<span style="color: blue;font-weight: bold; font-size: 40px;">PyCaret: ISBSG Data Analysis & Regression </span>


In [1]:
# <span style="color: blue;">ISBSG Data Analysis & Regression</span>


In [2]:
# # ISBSG Data Analysis and Regression Modeling
# 
# This notebook performs data cleaning, preprocessing, and regression modeling on the ISBSG dataset.

# ## Setup and Environment Configuration

# Install required packages (uncomment if needed)
!pip install -r requirements.txt

In [3]:
# Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycaret
import seaborn as sns
from datetime import datetime
import re
import sklearn

ModuleNotFoundError: No module named 'pycaret'

<a id = 'Index:'></a>

# Table of Content

In this notebook you will apply xxxxxxx


- [Part 1](#part1)- Data Loading and Initial Exploration
- [Part 2](#part2)- Data Cleaning and Preprocessing
- [Part 3](#part3)- Data Profiling
- [Part 4](#part4)- Module Building with PyCaret
- [Part 5](#part5)- Model Preparation
- [Part 6](#part6)- Baseline Modeling and Evaluation
- [Part 7](#part7)- Advanced Modeling and Hyperparameter Tuning
- [Part 8](#part8)- Model Comparison and Selection
- [Part 9](#part9)- End


In [ ]:
# Configure timestamp callback for Jupyter cells
from IPython import get_ipython

def setup_timestamp_callback():
    """Setup a timestamp callback for Jupyter cells without clearing existing callbacks."""
    ip = get_ipython()
    if ip is not None:
        # Define timestamp function
        def print_timestamp(*args, **kwargs):
            """Print timestamp after cell execution."""
            print(f"Cell executed at: {datetime.now()}")
        
        # Check if our callback is already registered
        callbacks = ip.events.callbacks.get('post_run_cell', [])
        for cb in callbacks:
            if hasattr(cb, '__name__') and cb.__name__ == 'print_timestamp':
                # Already registered
                return
                
        # Register new callback if not already present
        ip.events.register('post_run_cell', print_timestamp)
        print("Timestamp printing activated.")
    else:
        print("Not running in IPython/Jupyter environment.")

# Setup timestamp callback
setup_timestamp_callback()

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

[Back to top](#Index:)

<a id='part1'></a>

# Part 1 -Data Loading and Initial Exploration

This section is dedicated to loading the dataset, performing initial data exploration such as viewing the first few rows, and summarizing the dataset's characteristics, including missing values and basic statistical measures.

In [ ]:
# Load the data

from pathlib import Path

print("Loading data...")

file_path = "../data/ISBSG2016R1_1_FormattedForCSV_cleaned.csv"
file_name_no_ext = Path(file_path).stem                # 'ISBSG2016R1.1 - FormattedForCSV'
print(file_name_no_ext)


df = pd.read_csv(file_path)


In [ ]:
def display_header(text):
    try:
        from IPython.display import display, Markdown
        display(Markdown(f"# {text}"))
    except ImportError:
        print(f"\n=== {text} ===\n")

def display_subheader(text):
    try:
        from IPython.display import display, Markdown
        display(Markdown(f"## {text}"))
    except ImportError:
        print(f"\n-- {text} --\n")

def explore_data(df: pd.DataFrame) -> None:
    """
    Perform exploratory data analysis on the input DataFrame with nicely aligned plots.
    Args:
        df: Input DataFrame
    """
    from IPython.display import display

    display_header("Exploratory Data Analysis")
    
    # Data Overview
    display_subheader("Data Overview")
    print(f"Dataset shape: {df.shape}")
    if df.shape[0] > 20:
        print("First 5 rows:")
        display(df.head())
        print("Last 5 rows:")
        display(df.tail())
    else:
        display(df)
    
    # Duplicate Row Checking
    display_subheader("Duplicate Rows")
    num_duplicates = df.duplicated().sum()
    print(f"Number of duplicate rows: {num_duplicates}")

    # Data Types and Memory Usage
    display_subheader("Data Types and Memory Usage")
    dtype_info = pd.DataFrame({
        'Data Type': df.dtypes,
        'Memory Usage (MB)': df.memory_usage(deep=True) / 1024 / 1024
    })
    display(dtype_info)
    
    # Unique Values Per Column
    display_subheader("Unique Values Per Column")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()} unique values")
    
    # Type Conversion Suggestions
    display_subheader("Type Conversion Suggestions")
    potential_cat = [
        col for col in df.select_dtypes(include=['object']).columns
        if df[col].nunique() < max(30, 0.05*df.shape[0])
    ]
    if potential_cat:
        print("Consider converting to 'category' dtype for memory/performance:")
        print(potential_cat)
    else:
        print("No obvious candidates for 'category' dtype conversion.")
    
    # Summary Statistics
    display_subheader("Summary Statistics")
    try:
        display(df.describe(include='all').T.style.background_gradient(cmap='Blues', axis=1))
    except Exception:
        display(df.describe(include='all').T)
    
    # Missing Values
    display_subheader("Missing Values")
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_info = pd.DataFrame({
        'Missing Values': missing,
        'Percentage (%)': missing_percent.round(2)
    })
    if missing.sum() > 0:
        display(missing_info[missing_info['Missing Values'] > 0]
                .sort_values('Missing Values', ascending=False)
                .style.background_gradient(cmap='Reds'))
        # Visualize missing values
        plt.figure(figsize=(12, 6))
        cols_with_missing = missing_info[missing_info['Missing Values'] > 0].index
        if len(cols_with_missing) > 0:
            sns.heatmap(df[cols_with_missing].isnull(), 
                        cmap='viridis', 
                        yticklabels=False, 
                        cbar_kws={'label': 'Missing Values'})
            plt.title('Missing Value Patterns')
            plt.tight_layout()
            plt.show()
    else:
        print("No missing values in the dataset.")
    
    # Numerical Distributions
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    if len(numerical_cols) > 0:
        display_subheader("Distribution of Numerical Features")
        sample_cols = numerical_cols[:min(12, len(numerical_cols))]
        num_cols = len(sample_cols)
        num_rows = (num_cols + 2) // 3  # 3 plots per row, rounded up
        fig = plt.figure(figsize=(18, num_rows * 4))
        grid = plt.GridSpec(num_rows, 3, figure=fig, hspace=0.4, wspace=0.3)
        for i, col in enumerate(sample_cols):
            row, col_pos = divmod(i, 3)
            ax = fig.add_subplot(grid[row, col_pos])
            sns.histplot(df[col].dropna(), kde=True, ax=ax, color='skyblue', alpha=0.7)
            mean_val = df[col].mean()
            median_val = df[col].median()
            ax.axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.2f}')
            ax.axvline(median_val, color='green', linestyle=':', label=f'Median: {median_val:.2f}')
            stats_text = (f"Std: {df[col].std():.2f}\n"
                          f"Min: {df[col].min():.2f}\n"
                          f"Max: {df[col].max():.2f}")
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, fontsize=9,
                    verticalalignment='top', bbox=props)
            ax.set_title(f'Distribution of {col}')
            ax.legend(fontsize='small')
        plt.tight_layout()
        plt.show()
        # Correlation matrix and top correlations
        if len(numerical_cols) > 1:
            display_subheader("Correlation Matrix")
            corr = df[numerical_cols].corr().round(2)
            mask = np.triu(np.ones_like(corr, dtype=bool))
            plt.figure(figsize=(12, 10))
            sns.heatmap(corr, mask=mask, annot=True, cmap='coolwarm', 
                        fmt=".2f", linewidths=0.5, vmin=-1, vmax=1, 
                        annot_kws={"size": 10})
            plt.title('Correlation Matrix (Lower Triangle Only)', fontsize=14)
            plt.xticks(rotation=45, ha='right', fontsize=10)
            plt.yticks(fontsize=10)
            plt.tight_layout()
            plt.show()
            # Top correlations
            if len(numerical_cols) > 5:
                corr_unstack = corr.unstack()
                corr_abs = corr_unstack.apply(abs)
                corr_abs = corr_abs[corr_abs < 1.0]
                highest_corrs = corr_abs.sort_values(ascending=False).head(15)
                display_subheader("Top Correlations")
                for (col1, col2), corr_val in highest_corrs.items():
                    actual_val = corr.loc[col1, col2]
                    print(f"{col1} — {col2}: {actual_val:.2f}")
                pairs_to_plot = [(idx[0], idx[1]) for idx in highest_corrs.index][:6]
                if pairs_to_plot:
                    fig = plt.figure(figsize=(18, 12))
                    grid = plt.GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)
                    for i, (col1, col2) in enumerate(pairs_to_plot):
                        row, col_pos = divmod(i, 3)
                        ax = fig.add_subplot(grid[row, col_pos])
                        sns.regplot(x=df[col1], y=df[col2], ax=ax, scatter_kws={'alpha':0.5})
                        r_value = df[col1].corr(df[col2])
                        ax.set_title(f'{col1} vs {col2} (r = {r_value:.2f})')
                    plt.tight_layout()
                    plt.show()
    # Categorical columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    if len(categorical_cols) > 0:
        display_subheader("Categorical Features")
        sample_cat_cols = categorical_cols[:min(6, len(categorical_cols))]
        num_cat_cols = len(sample_cat_cols)
        num_cat_rows = (num_cat_cols + 1) // 2
        fig = plt.figure(figsize=(18, num_cat_rows * 5))
        grid = plt.GridSpec(num_cat_rows, 2, figure=fig, hspace=0.4, wspace=0.2)
        for i, col in enumerate(sample_cat_cols):
            row, col_pos = divmod(i, 2)
            ax = fig.add_subplot(grid[row, col_pos])
            value_counts = df[col].value_counts().sort_values(ascending=False)
            top_n = min(10, len(value_counts))
            if len(value_counts) > top_n:
                top_values = value_counts.head(top_n-1)
                other_count = value_counts.iloc[top_n-1:].sum()
                plot_data = pd.concat([top_values, pd.Series({'Other': other_count})])
            else:
                plot_data = value_counts
            sns.barplot(x=plot_data.values, y=plot_data.index, ax=ax, palette='viridis')
            ax.set_title(f'Distribution of {col} (Total: {len(value_counts)} unique values)')
            ax.set_xlabel('Count')
            total = plot_data.sum()
            for j, v in enumerate(plot_data.values):
                percentage = v / total * 100
                ax.text(v + 0.1, j, f'{percentage:.1f}%', va='center')
        plt.tight_layout()
        plt.show()
        # Categorical-numerical boxplots
        if numerical_cols and len(categorical_cols) > 0:
            display_subheader("Categorical-Numerical Relationships")
            numerical_variances = df[numerical_cols].var()
            target_numerical = numerical_variances.idxmax()
            sample_cat_for_box = [col for col in categorical_cols 
                                  if df[col].nunique() <= 15][:4]
            if sample_cat_for_box:
                fig = plt.figure(figsize=(18, 5 * len(sample_cat_for_box)))
                for i, cat_col in enumerate(sample_cat_for_box):
                    ax = fig.add_subplot(len(sample_cat_for_box), 1, i+1)
                    order = df.groupby(cat_col)[target_numerical].median().sort_values().index
                    sns.boxplot(x=cat_col, y=target_numerical, data=df, ax=ax, 
                                order=order, palette='Set3')
                    ax.set_title(f'{cat_col} vs {target_numerical}')
                    ax.set_xlabel(cat_col)
                    ax.set_ylabel(target_numerical)
                    plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.show()

# Exploratory Data Analysis
explore_data(df)


In [ ]:
# Clean column names function
def clean_column_names(columns):
    cleaned_cols = []
    for col in columns:
        # First replace ampersands with _&_ to match PyCaret's transformation
        col_clean = col.replace(' & ', '_&_')
        # Then remove any remaining special chars
        col_clean = re.sub(r'[^\w\s&]', '', col_clean)
        # Finally replace spaces with underscores
        col_clean = col_clean.replace(' ', '_')
        cleaned_cols.append(col_clean)
    return cleaned_cols

# Clean column names
original_columns = df.columns.tolist()  # Save original column names for reference
df.columns = clean_column_names(df.columns)

In [ ]:
# Create a mapping from original to cleaned column names
column_mapping = dict(zip(original_columns, df.columns))
print("\nColumn name mapping (original -> cleaned):")
for orig, clean in column_mapping.items():
    if orig != clean:  # Only show columns that changed
        print(f"  '{orig}' -> '{clean}'")


In [ ]:
# Display basic information
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())


In [ ]:
# Create a function to get comprehensive data summary
def get_data_summary(df, n_unique_samples=5):
    """
    Generate a comprehensive summary of the dataframe.
    
    Args:
        df: Pandas DataFrame
        n_unique_samples: Number of unique values to show as sample
        
    Returns:
        DataFrame with summary information
    """
    # Summary dataframe with basic info
    summary = pd.DataFrame({
        'Feature': df.columns,
        'data_type': df.dtypes.values,
        'Null_number': df.isnull().sum().values,
        'Null_pct': (df.isnull().mean() * 100).values,
        'Unique_counts': df.nunique().values,
        'unique_samples': [list(df[col].dropna().unique()[:n_unique_samples]) for col in df.columns]
    })
    
    return summary

# Generate and display data summary
summary_df = get_data_summary(df)
print("\nData Summary (first 10 columns):")
print(summary_df.head(10))


In [ ]:
# Identify target column
target_col = 'project_prf_normalised_work_effort'
print(f"\nTarget variable: '{target_col}'")

[Back to top](#Index:)

<a id='part2'></a>

# Part 2 - Data Cleaning and Preprocessing

Here, data cleaning tasks like handling missing values and providing a detailed summary of each feature, including its type, number of unique values, and a preview of unique values, are performed.

In [ ]:
# Analyse missing values
print("\nAnalysing missing values...")
missing_pct = df.isnull().mean() * 100
missing_sorted = missing_pct.sort_values(ascending=False)
print("Top 10 columns with highest missing percentages:")
print(missing_sorted.head(10))

In [ ]:
# Identify columns with high missing values (>70%)
high_missing_cols = missing_pct[missing_pct > 70].index.tolist()
print(f"\nColumns with >70% missing values ({len(high_missing_cols)} columns):")
for col in high_missing_cols[:5]:  # Show first 5
    print(f"  - {col}: {missing_pct[col]:.2f}% missing")
if len(high_missing_cols) > 5:
    print(f"  - ... and {len(high_missing_cols) - 5} more columns")

In [ ]:
# Create a clean dataframe by dropping high-missing columns
df_clean = df.drop(columns=high_missing_cols)
print(f"\nData shape after dropping high-missing columns: {df_clean.shape}")
print(f"\nHigh missing columns got dropped are: {high_missing_cols}")

# Numerical columns
num_cols = df_clean.select_dtypes(include=['number']).columns.tolist()
print("\nNumerical columns:")
print(num_cols)

# Categorical columns (object or category dtype)
cat_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()
print("\nCategorical columns:")
print(cat_cols)



In [ ]:
# Handle remaining missing values
print("\nHandling remaining missing values...")

In [ ]:
# Fill missing values in categorical columns with "Missing"
cat_cols = df_clean.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    df_clean[col].fillna('Missing', inplace=True)

In [ ]:
# Check remaining missing values
remaining_missing = df_clean.isnull().sum()
remaining_missing_count = sum(remaining_missing > 0)
print(f"\nColumns with remaining missing values: {remaining_missing_count}")
if remaining_missing_count > 0:
    print("Top columns with missing values:")
    print(remaining_missing[remaining_missing > 0].sort_values(ascending=False).head())

In [ ]:
# Verify target variable
print(f"\nTarget variable '{target_col}' summary:")
print(f"Unique values: {df_clean[target_col].nunique()}")
print(f"Missing values: {df_clean[target_col].isnull().sum()}")
print(f"Top value counts:")
print(df_clean[target_col].value_counts().head())


In [ ]:
# Check for infinite values
inf_check = np.isinf(df_clean.select_dtypes(include=[np.number])).sum().sum()
print(f"\nNumber of infinite values: {inf_check}")

In [ ]:
# Save cleaned data

file_name_no_ext

df_clean.to_csv(f'../data/{file_name_no_ext}_dropped.csv', index=False)
print(f'../data/{file_name_no_ext}_dropped.csv')


[Back to top](#Index:)

<a id='part3'></a>

# Part 3 - Feature Engineering and Selection

Involves creating or selecting specific features for the model based on insights from EDA, including handling categorical variables and reducing dimensionality if necessary.

In [ ]:
# Identify categorical columns and check cardinality
print("\nCategorical columns and their cardinality:")
cat_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()
for col in cat_cols[:5]:  # Show first 5
    print(f"  {col}: {df_clean[col].nunique()} unique values")
if len(cat_cols) > 5:
    print(f"  ... and {len(cat_cols) - 5} more columns")

In [ ]:
# One-hot encode categorical columns with low cardinality (<10 unique values)
low_card_cols = [col for col in cat_cols if df_clean[col].nunique() < 10]
print(f"\nApplying one-hot encoding to {len(low_card_cols)} low-cardinality columns:")
for col in low_card_cols[:5]:  # Show first 5
    print(f"  - {col}")
if len(low_card_cols) > 5:
    print(f"  - ... and {len(low_card_cols) - 5} more columns")


In [ ]:
# Create encoded dataframe
df_encoded = pd.get_dummies(df_clean, columns=low_card_cols, drop_first=True)
print(f"\nData shape after one-hot encoding: {df_encoded.shape}")
print("\nAll column names:")
print(df_encoded.columns.tolist())


In [ ]:
# MANUALLY fix the problematic column names BEFORE PyCaret setup

# Function to fix the column names and prevent duplicates
def fix_column_names_no_duplicates(df):
    """Fix column names that cause issues with PyCaret while preventing duplicates."""
    original_cols = df.columns.tolist()
    fixed_columns = []
    
    # Track columns to check for duplicates
    seen_columns = set()
    
    for col in original_cols:
        # Replace spaces with underscores
        fixed_col = col.replace(' ', '_')
        # Replace ampersands 
        fixed_col = fixed_col.replace('&', 'and')
        # Remove any other problematic characters
        fixed_col = ''.join(c if c.isalnum() or c == '_' else '_' for c in fixed_col)
        
        # Handle duplicates by appending a suffix
        base_col = fixed_col
        suffix = 1
        while fixed_col in seen_columns:
            fixed_col = f"{base_col}_{suffix}"
            suffix += 1
        
        seen_columns.add(fixed_col)
        fixed_columns.append(fixed_col)
    
    # Create a new DataFrame with fixed column names
    df_fixed = df.copy()
    df_fixed.columns = fixed_columns
    
    # Print statistics about the renaming
    n_changed = sum(1 for old, new in zip(original_cols, fixed_columns) if old != new)
    print(f"Changed {n_changed} column names.")
    
    # Check for duplicates in the new column names
    dup_check = [item for item, count in pd.Series(fixed_columns).value_counts().items() if count > 1]
    if dup_check:
        print(f"WARNING: Found {len(dup_check)} duplicate column names after fixing: {dup_check}")
    else:
        print("No duplicate column names in the fixed DataFrame.")
    
    return df_fixed

# Show some of the original column names to help diagnose issues
print("\nSample of original column names:")
for i, col in enumerate(df_encoded.columns[:15]):  # Show first 15 for diagnosis
    print(f"{i}: {col}")

# Apply the fix to your dataframe
print("\nFixing column names for PyCaret compatibility...")
df_fixed = fix_column_names_no_duplicates(df_encoded)

# Print some example fixed columns to verify
print("\nSample of fixed column names:")
for i, (old, new) in enumerate(zip(df_encoded.columns[:15], df_fixed.columns[:15])):
    print(f"Original: {old} -> Fixed: {new}")

In [ ]:
# Save this DataFrame with fixed column names

df_fixed.to_csv(f'../data/{file_name_no_ext}_fixed_columns_data.csv', index=False)
print(f"Saved data with fixed column names to '../data/{file_name_no_ext}_fixed_columns_data.csv'")

In [ ]:
# Create a diagnostic file with all column transformations
with open('../temp/column_transformations.txt', 'w') as f:
    f.write("Column name transformations:\n")
    f.write("--------------------------\n")
    for old, new in zip(df_encoded.columns, df_fixed.columns):
        f.write(f"{old} -> {new}\n")
print("Saved complete column transformations to '../temp/column_transformations.txt'")

[Back to top](#Index:)

<a id='part4'></a>

# Part 4 - Data Profiling

xxx

In [ ]:
# ## Data Profiling (Optional)

try:
    from ydata_profiling import ProfileReport
    
    print("\nGenerating data profile report...")
    profile = ProfileReport(df_clean, title="ISBSG Dataset Profiling Report", minimal=True)
    profile.to_file("data_profile.html")
    print("Data profile report saved to 'data_profile.html'")
except ImportError:
    print("\nSkipping data profiling (ydata_profiling not installed)")
    print("To install: pip install ydata-profiling")

[Back to top](#Index:)

<a id='part5'></a>

# Part 5 - PyCaret setup

xxx

In [ ]:
print(sklearn.__version__)
print(pycaret.__version__)  

In [ ]:
from pycaret.regression import setup, get_config

ignore_cols = ['isbsg_project_id', 'external_eef_data_quality_rating',  'project_prf_normalised_level_1_pdr_ufp', 'project_prf_normalised_pdr_ufp', 
               'project_prf_project_elapsed_time', 'people_prf_ba_team_experience_less_than_1_yr', 'people_prf_ba_team_experience_1_to_3_yr', 
               'people_prf_ba_team_experience_great_than_3_yr', 'people_prf_it_experience_less_than_1_yr', 'people_prf_it_experience_1_to_3_yr', 
               'people_prf_it_experience_great_than_3_yr', 'people_prf_it_experience_less_than_3_yr', 'people_prf_it_experience_3_to_9_yr', 
               'people_prf_it_experience_great_than_9_yr', 'people_prf_project_manage_experience', 'project_prf_total_project_cost', 
               'project_prf_cost_currency', 'project_prf_currency_multiple', 'project_prf_speed_of_delivery', 'people_prf_project_manage_changes', 
               'project_prf_defect_density','project_prf_manpower_delivery_rate'
            ]
setup_results = setup(
    data=df_fixed,
    target=target_col,
    ignore_features=ignore_cols,
    session_id=123,
    preprocess=True,
    # ...other options...
    verbose=False
)

# Get the fitted pipeline from PyCaret
preprocessor = get_config('preprocessor')
# The scaler is usually inside the pipeline's steps
for name, step in preprocessor.named_steps.items():
    if 'scaler' in name:
        scaler = step
        break

import pickle
with open('../models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


[Back to top](#Index:)

<a id='part6'></a>

# Part 6 - Feature Correlation Analysis

xxx

In [ ]:
# Feature correlation analysis
print("\nAnalyzing feature correlations...")
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os
    from pycaret.regression import get_config

    # Create directory for plots
    os.makedirs('plots', exist_ok=True)

    # Get data from PyCaret
    X = get_config('X')

    # Ensure we're working with numeric data only
    X_numeric = X.select_dtypes(include=[np.number])

    # Drop rows with NaN or Inf values before correlation and VIF analysis
    X_numeric_clean = X_numeric.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how='any')

    # Get number of features
    n_features = X_numeric_clean.shape[1]
    print(f"Analyzing correlations among {n_features} numeric features")

    # Calculate correlation matrix
    corr_matrix = X_numeric_clean.corr()

    # Determine features with high correlation
    correlation_threshold = 0.7
    high_corr_pairs = []

    # Find highly correlated feature pairs
    for i in range(n_features):
        for j in range(i+1, n_features):
            if abs(corr_matrix.iloc[i, j]) > correlation_threshold:
                high_corr_pairs.append((
                    X_numeric_clean.columns[i],
                    X_numeric_clean.columns[j],
                    corr_matrix.iloc[i, j]
                ))

    # Plot correlation heatmap
    plt.figure(figsize=(14, 12))
    mask = np.triu(corr_matrix)
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # If there are too many features, show only the ones with high correlation
    if n_features > 20:
        print(f"Large number of features detected ({n_features}). Creating filtered correlation matrix.")
        # Get list of features with high correlation
        high_corr_features = set()
        for feat1, feat2, _ in high_corr_pairs:
            high_corr_features.add(feat1)
            high_corr_features.add(feat2)

        # If there are high correlations, show only those features
        if high_corr_features:
            high_corr_features = list(high_corr_features)
            filtered_corr = corr_matrix.loc[high_corr_features, high_corr_features]

            # Plot filtered heatmap
            sns.heatmap(filtered_corr, mask=np.triu(filtered_corr),
                        cmap=cmap, vmax=1, vmin=-1, center=0,
                        square=True, linewidths=.5, cbar_kws={"shrink": .5},
                        annot=True, fmt=".2f")
            plt.title('Correlation Heatmap (Filtered to Highly Correlated Features)')
        else:
            # No high correlations, show full matrix
            sns.heatmap(corr_matrix, mask=mask,
                        cmap=cmap, vmax=1, vmin=-1, center=0,
                        square=True, linewidths=.5, cbar_kws={"shrink": .5})
            plt.title('Correlation Heatmap (All Features)')
    else:
        # For smaller feature sets, show the full correlation matrix
        sns.heatmap(corr_matrix, mask=mask,
                    cmap=cmap, vmax=1, vmin=-1, center=0,
                    square=True, linewidths=.5, cbar_kws={"shrink": .5},
                    annot=True, fmt=".2f")
        plt.title('Correlation Heatmap (All Features)')

    plt.tight_layout()
    plt.savefig('plots/correlation_heatmap.png')
    plt.show()      # <-- Show in notebook
    plt.close()
    print("Correlation heatmap saved as plots/correlation_heatmap.png")

    # Calculate Variance Inflation Factor (VIF) if there are enough samples
    vif_data = None
    if X_numeric_clean.shape[0] > X_numeric_clean.shape[1]:
        try:
            from statsmodels.stats.outliers_influence import variance_inflation_factor

            # Calculate VIF for each feature
            vif_data = pd.DataFrame()
            vif_data["Feature"] = X_numeric_clean.columns
            vif_data["VIF"] = [variance_inflation_factor(X_numeric_clean.values, i)
                               for i in range(X_numeric_clean.shape[1])]

            # Sort by VIF value
            vif_data = vif_data.sort_values("VIF", ascending=False)

            # Plot VIF values
            plt.figure(figsize=(12, 8))
            plt.barh(vif_data["Feature"], vif_data["VIF"])
            plt.axvline(x=5, color='r', linestyle='--', label='VIF=5 (Moderate multicollinearity)')
            plt.axvline(x=10, color='darkred', linestyle='--', label='VIF=10 (High multicollinearity)')
            plt.xlabel('VIF Value')
            plt.title('Variance Inflation Factor (VIF) for Features')
            plt.legend()
            plt.tight_layout()
            plt.savefig('plots/vif_values.png')
            plt.show()      # <-- Show in notebook
            plt.close()
            print("VIF values plot saved as plots/vif_values.png")
        except Exception as vif_err:
            print(f"Could not calculate VIF: {vif_err}")
    else:
        print("Not enough samples to calculate VIF (need more samples than features)")

    # Print results
    print(f"\nFound {len(high_corr_pairs)} feature pairs with correlation > {correlation_threshold}:")
    for feat1, feat2, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True):
        print(f"  • {feat1} and {feat2}: {corr:.4f}")

    # Print VIF results if available
    if vif_data is not None:
        high_vif_threshold = 10
        high_vif_features = vif_data[vif_data["VIF"] > high_vif_threshold]
        if not high_vif_features.empty:
            print(f"\nFeatures with high VIF (> {high_vif_threshold}):")
            for _, row in high_vif_features.iterrows():
                print(f"  • {row['Feature']}: {row['VIF']:.2f}")
        else:
            print(f"\nNo features have VIF > {high_vif_threshold}")

    # Recommendations based on analysis
    print("\n--- Multicollinearity Analysis Recommendations ---")
    if high_corr_pairs:
        print("Consider addressing multicollinearity by:")
        print("1. Removing one feature from each highly correlated pair")
        print("2. Creating new features by combining correlated features")
        print("3. Applying dimensionality reduction techniques like PCA")

        # Identify top candidates for removal
        if len(high_corr_pairs) > 0:
            print("\nPotential candidates for removal:")
            # Count frequency of each feature in high correlation pairs
            freq = {}
            for feat1, feat2, _ in high_corr_pairs:
                freq[feat1] = freq.get(feat1, 0) + 1
                freq[feat2] = freq.get(feat2, 0) + 1

            # Features that appear most frequently in high correlation pairs
            freq_df = pd.DataFrame({'Feature': list(freq.keys()),
                                    'Frequency in high corr pairs': list(freq.values())})
            freq_df = freq_df.sort_values('Frequency in high corr pairs', ascending=False)

            for _, row in freq_df.head(5).iterrows():
                print(f"  • {row['Feature']} (appears in {row['Frequency in high corr pairs']} high correlation pairs)")
    else:
        print("No significant multicollinearity detected based on correlation analysis.")

    if vif_data is not None and not high_vif_features.empty:
        print("\nBased on VIF analysis, consider removing or transforming these features with high VIF values.")

except Exception as e:
    print(f"Feature correlation analysis failed: {e}")


[Back to top](#Index:)

<a id='part7'></a>

# Part 7 - Model Building with PyCaret

xxx

In [ ]:
from pycaret.regression import get_config, compare_models, pull, tune_model, evaluate_model, save_model

import time

# Start timing
start_time = time.time()

# Create output directories if needed
os.makedirs('data', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('logs', exist_ok=True)


# Get preprocessed data for inspection and saving
X = get_config("X")
y = get_config("y")
X.to_csv('data/pycaret_processed_features.csv', index=False)
y.to_csv('data/pycaret_processed_target.csv', index=False)
print(f"\nPreprocessed data shape: {X.shape}")
print(f"Numeric features: {len(X.select_dtypes(include=[float, int]).columns)}")
print(f"Categorical features: {len(X.select_dtypes(include=['object', 'category']).columns)}")
print("Preprocessed features and target saved.")


# 1. Compare and select top 3 models (returns list of models)
print("\nComparing regression models and selecting top 3...")
top_models = compare_models(n_select=3)
model_results = pull()
model_results.to_csv('logs/model_comparison_results.csv')
print("\nModel comparison results:")
print(model_results)

# 2. For each top model: tune, evaluate, and save
tuned_models = []
scores = []

for i, model in enumerate(top_models, 1):
    model_name = type(model).__name__
    print(f"\nModel {i}: {model_name}")
    
    # Tune
    print("  Tuning...")
    tuned = tune_model(model, n_iter=10)
    tuned_models.append(tuned)

    # Pull results after tuning - get the mean values
    tuned_results = pull()
    tuned_results.to_csv(f'logs/tuned_results_model_{i}_{model_name}.csv')
    
    # Extract metrics from "Mean" column instead of "Value"
    try:
        # First try to access by 'Mean' column which is the typical format
        scores.append({
            'Model': model_name, 
            'MAE': tuned_results.loc['MAE', 'Mean'],
            'RMSE': tuned_results.loc['RMSE', 'Mean'],
            'R2': tuned_results.loc['R2', 'Mean']
        })
    except KeyError:
        # As a fallback, check the structure of tuned_results
        print(f"  Warning: Expected column structure not found in tuned results")
        print(f"  tuned_results columns: {tuned_results.columns}")
        print(f"  tuned_results index: {tuned_results.index}")
        
        # Try alternative approaches based on the actual structure
        if 'Mean' in tuned_results.columns:
            scores.append({
                'Model': model_name,
                'MAE': tuned_results.loc['MAE', 'Mean'] if 'MAE' in tuned_results.index else None,
                'RMSE': tuned_results.loc['RMSE', 'Mean'] if 'RMSE' in tuned_results.index else None,
                'R2': tuned_results.loc['R2', 'Mean'] if 'R2' in tuned_results.index else None
            })
        elif len(tuned_results.columns) > 0:
            # Get the last column as it might contain mean values
            last_col = tuned_results.columns[-1]
            scores.append({
                'Model': model_name,
                'MAE': tuned_results.loc['MAE', last_col] if 'MAE' in tuned_results.index else None,
                'RMSE': tuned_results.loc['RMSE', last_col] if 'RMSE' in tuned_results.index else None,
                'R2': tuned_results.loc['R2', last_col] if 'R2' in tuned_results.index else None
            })
        else:
            # If we still can't find the right structure, log the issue
            scores.append({
                'Model': model_name,
                'MAE': None,
                'RMSE': None,
                'R2': None
            })
            print(f"  Unable to extract metrics for {model_name}. Check the saved CSV for details.")
    
    # Save tuned model
    save_model(tuned, f'models/top_model_{i}_{model_name}')
    print(f"  Saved as models/top_model_{i}_{model_name}.pkl")
    print(f"  Time elapsed: {time.time() - start_time:.1f} seconds")


# Save overall summary of all tuned models
score_df = pd.DataFrame(scores)
score_df.to_csv('logs/tuned_model_scores.csv', index=False)
print("\nTuned models summary:\n", score_df)
print("\nAll top 3 models have been tuned, evaluated, and saved.")
print("\nAnalysis complete! Proceed with feature importance or SHAP analysis as next steps.")

# 3. Optionally: Pull the best model for additional analysis (feature importance, SHAP, etc.)
# You can access the best model as top_models[0] or reload any saved model later



[Back to top](#Index:)

<a id='part8'></a>

# Part 8 - Feature Importance

xxx

In [ ]:
# Only some models (e.g., tree-based models like Random Forest, XGBoost, LightGBM) have feature_importances_.
# Many linear models (like LinearRegression, Lasso), KNN, and some ensemble models do not.

# print(type(tuned_model))

for i, m in enumerate(tuned_models, 1):
    print(f"Model {i} type: {type(m)}")



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
import pandas as pd

def plot_linear_feature_importance(model, X, y, feature_names=None, method='coefficients'):
    """
    Plot feature importance for Linear models
    
    Parameters:
    -----------
    model : trained linear model
    X : feature matrix
    y : target vector
    feature_names : list of feature names (optional)
    method : 'coefficients' or 'permutation'
    """
    
    # Create directory for plots if it doesn't exist
    import os
    os.makedirs('plots', exist_ok=True)
    
    # Get feature names if not provided
    if feature_names is None:
        if hasattr(X, 'columns'):  # If X is a DataFrame
            feature_names = X.columns.tolist()
        else:
            feature_names = [f'Feature {i}' for i in range(X.shape[1])]
    
    plt.figure(figsize=(10, 6))
    
    if method == 'coefficients':
        # Use absolute coefficient values as feature importance
        importances = np.abs(model.coef_)
        indices = np.argsort(importances)
        
        plt.title('Feature Importance Based on Coefficient Magnitude')
        plt.barh(range(len(indices)), importances[indices], align='center')
        plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
        plt.xlabel('Absolute Coefficient Magnitude')
        
    elif method == 'permutation':
        # Calculate permutation importance
        result = permutation_importance(
            model, X, y, n_repeats=10, random_state=42, n_jobs=-1
        )
        
        importances = result.importances_mean
        std = result.importances_std
        indices = np.argsort(importances)
        
        plt.title('Feature Importance Based on Permutation Importance')
        plt.barh(range(len(indices)), importances[indices], xerr=std[indices], align='center')
        plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
        plt.xlabel('Permutation Importance')
    
    plt.tight_layout()
    plt.savefig(f'plots/linear_feature_importance_{method}.png')
    print(f'Feature importance plot saved to plots/linear_feature_importance_{method}.png')
    
    # Return the importances for potential further analysis
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    return importance_df

# Example usage:
# plot_linear_feature_importance(linear_model, X, y, feature_names=X.columns, method='coefficients')
# plot_linear_feature_importance(linear_model, X, y, feature_names=X.columns, method='permutation')

In [ ]:
# code modified for top 3 models

from pycaret.regression import plot_model
import os
import matplotlib.pyplot as plt

pycaret_X=X
pycaret_y = y

os.makedirs('plots', exist_ok=True)
feature_names = pycaret_X.columns.tolist()  # Make sure to use the same data as in training

for i, tuned_model in enumerate(tuned_models, 1):
    model_name = type(tuned_model).__name__
    print(f"\nModel {i}: {model_name}")

    # First try PyCaret's plot_model
    try:
        plot_model(tuned_model, plot='feature', save=False)
        plt.savefig(f'plots/feature_importance_model_{i}_{model_name}.png')
        plt.show()
        plt.close()
        print(f"  PyCaret feature importance plot saved to plots/feature_importance_model_{i}_{model_name}.png")
    except Exception as e:
        print(f"  PyCaret plot_model failed: {e}")
        # Fallback for linear models with coefficients
        try:
            # If it's a linear model (like HuberRegressor, LinearRegression, etc.)
            if hasattr(tuned_model, 'coef_'):
                importance_df = plot_linear_feature_importance(
                    tuned_model, pycaret_X, pycaret_y, 
                    feature_names=feature_names, 
                    method='coefficients'
                )
                print("  Custom coefficient-based feature importance plot saved.")
                print("  Top 5 important features:")
                print(importance_df.head(5))
            else:
                print("  This model does not support .coef_ or is not a linear model.")
        except Exception as e2:
            print(f"  Could not generate feature plot for linear model: {e2}")

    # Optionally: also plot permutation-based feature importance for all linear models
    if hasattr(tuned_model, 'coef_'):
        print("\n  Generating permutation-based feature importance plot...")
        try:
            importance_df_perm = plot_linear_feature_importance(
                tuned_model, pycaret_X, pycaret_y, 
                feature_names=feature_names, 
                method='permutation'
            )
            print("  Top 5 important features (permutation):")
            print(importance_df_perm.head(5))
        except Exception as e:
            print(f"  Could not generate permutation feature plot: {e}")


[Back to top](#Index:)

<a id='part9'></a>

# Part 9 - SHAP Analysis

xxx

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Check if required packages are installed and install if needed
def check_and_install_packages():
    try:
        import shap
        from pycaret.regression import get_config
        print("All required packages are installed.")
        return True
    except ImportError as e:
        missing_package = str(e).split("'")[1]
        print(f"Missing package: {missing_package}")
        install = input(f"Would you like to install {missing_package}? (y/n): ")
        if install.lower() == 'y':
            import sys
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", missing_package])
            print(f"{missing_package} installed successfully.")
            return True
        else:
            print(f"⚠️ Please install {missing_package} to proceed.")
            return False

In [ ]:
# SHAP analysis with proper data type handling and debugging
"""
Global Perspective:
- Summary Plot: Provides a global overview of feature importance and their 
  positive or negative impact on the model output across the entire dataset.
- Dependence Plot: Illustrates the relationship between a single feature's 
  value and its SHAP value across all instances to understand its general 
  effect on the prediction.
- Bar Chart: Shows the global importance of each feature based on the average 
  magnitude of their SHAP values across the entire dataset.

Single Instance Perspective:
- Force Plot: Explains the prediction for a single instance by showing how each 
  feature contributes to moving the prediction from the base value for that specific case.
- Waterfall Plot: Explains the prediction for a single instance by visualizing the sequential, 
  additive contribution of each feature's SHAP value for that specific prediction.
"""

def run_shap_analysis(tuned_model, model_index=1, debug=True):
    """
    Run SHAP analysis on a tuned model with enhanced debugging and all major SHAP plots.
    """
    print(f"\n{'='*50}")
    print(f"SHAP Analysis for Model {model_index}")
    print(f"{'='*50}")
    try:
        import shap
        import matplotlib.pyplot as plt
        import numpy as np
        import os

        os.makedirs('plots', exist_ok=True)
        if debug:
            print(f"Output directory created: {os.path.abspath('plots')}")

        try:
            X_transformed = get_config('X_transformed')
            if debug:
                print(f"Successfully retrieved transformed data")
                print(f"   Shape: {X_transformed.shape}")
                print(f"   Data types: {X_transformed.dtypes.value_counts()}")
                print(f"   Contains NaN: {np.isnan(X_transformed).any()}")
        except Exception as e:
            print(f"❌ Error getting transformed data: {e}")
            print("   Trying alternative approach...")
            try:
                X = get_config('X')
                X_transformed = X
                print(f"Using raw features instead. Shape: {X_transformed.shape}")
            except Exception as e2:
                print(f"❌ Error getting raw data: {e2}")
                raise ValueError("Could not access training data")
        
        model_name = type(tuned_model).__name__
        if debug:
            print(f"Model identified as: {model_name}")

        # Convert data to float64 for SHAP
        try:
            X_transformed_float = X_transformed.astype(np.float64)
            if debug:
                print("Data successfully converted to float64 type")
        except Exception as e:
            print(f"❌ Error converting data types: {e}")
            try:
                X_transformed_float = X_transformed.copy()
                for col in X_transformed.columns:
                    X_transformed_float[col] = pd.to_numeric(X_transformed[col], errors='coerce')
                X_transformed_float = X_transformed_float.fillna(0)
                print("Data converted using alternative method")
            except:
                print("❌ Could not convert data. Proceeding with original data.")
                X_transformed_float = X_transformed

        try:
            feature_names = get_config('X').columns.tolist()
            if debug:
                print(f"Retrieved {len(feature_names)} feature names")
        except:
            feature_names = [f"Feature_{i}" for i in range(X_transformed.shape[1])]
            print(f"⚠️ Could not get original feature names, using generic names")

        model_type = str(type(tuned_model)).lower()
        if debug:
            print(f"📊 Model type details: {model_type}")

        # Sample for efficiency
        sample_size = min(100, X_transformed_float.shape[0])
        sample_indices = np.random.choice(X_transformed_float.shape[0], sample_size, replace=False)
        X_sample = X_transformed_float.iloc[sample_indices] if hasattr(X_transformed_float, 'iloc') else X_transformed_float[sample_indices]

        # Choose SHAP explainer
        if any(x in model_type for x in ['tree', 'forest', 'xgboost', 'lgbm', 'catboost', 'gradientboosting']):
            print("🌲 Using TreeExplainer for tree-based model")
            explainer = shap.TreeExplainer(tuned_model)
            shap_values = explainer(X_sample)
        elif any(x in model_type for x in ['linear', 'logistic', 'ridge', 'lasso', 'huber']):
            print("📏 Using LinearExplainer for linear model")
            explainer = shap.LinearExplainer(tuned_model, X_sample)
            shap_values = explainer(X_sample)
        else:
            print("🔄 Using KernelExplainer as fallback (may be slow)")
            def model_predict(X):
                return tuned_model.predict(X)
            explainer = shap.KernelExplainer(model_predict, X_sample)
            shap_values = explainer.shap_values(X_sample)

        # Prepare shap_array for plotting
        shap_array = shap_values.values if hasattr(shap_values, "values") else shap_values

        print(f"shap_array shape: {getattr(shap_array, 'shape', 'N/A')}, X_sample shape: {X_sample.shape}")

        # ========== GLOBAL PERSPECTIVE ==========
        # 1. Summary Plot
        try:
            if isinstance(shap_array, np.ndarray) and shap_array.shape[1] == X_sample.shape[1]:
                plt.figure(figsize=(12, 10))
                shap.summary_plot(shap_array, X_sample, feature_names=feature_names, show=False)
                plt.tight_layout()
                summary_path = f'plots/shap_summary_model{model_index}_{model_name}.png'
                plt.savefig(summary_path)
                plt.show()  # This ensures it's visible inline
                plt.close()
                print(f"✅ SHAP summary plot saved to {os.path.abspath(summary_path)}")
            else:
                print(f"❌ SHAP values shape {shap_array.shape} does not match sample features {X_sample.shape}")
                print("Skipping summary plot.")
        except Exception as e:
            print(f"❌ Error creating summary plot: {e}")

        # 2. Dependence Plot (top global feature)
        try:
            # Top global feature by mean absolute SHAP value
            if isinstance(shap_array, np.ndarray) and shap_array.shape[1] == X_sample.shape[1]:
                top_idx = np.argsort(np.abs(shap_array).mean(0))[-1]
                top_feat = feature_names[top_idx]
                plt.figure(figsize=(10, 7))
                shap.dependence_plot(top_feat, shap_array, X_sample, feature_names=feature_names, show=False)
                dep_path = f'plots/shap_dependence_{top_feat}_model{model_index}_{model_name}.png'
                plt.tight_layout()
                plt.savefig(dep_path)
                plt.show()  # This ensures it's visible inline
                plt.close()
                print(f"✅ SHAP dependence plot for '{top_feat}' saved to {os.path.abspath(dep_path)}")
        except Exception as e:
            print(f"❌ Error creating dependence plot: {e}")

        # 3. Bar Chart (mean absolute SHAP values)
        try:
            if isinstance(shap_array, np.ndarray) and shap_array.shape[1] == X_sample.shape[1]:
                plt.figure(figsize=(12, 8))
                shap.summary_plot(shap_array, X_sample, feature_names=feature_names, plot_type="bar", show=False)
                plt.tight_layout()
                bar_path = f'plots/shap_importance_bar_model{model_index}_{model_name}.png'
                plt.savefig(bar_path)
                plt.show()  # This ensures it's visible inline
                plt.close()
                print(f"✅ SHAP feature importance bar plot saved to {os.path.abspath(bar_path)}")
        except Exception as e:
            print(f"❌ Error creating bar plot: {e}")

        # ========== SINGLE INSTANCE PERSPECTIVE ==========
        # Choose a single instance (first row of sample)
        instance = X_sample.iloc[0] if hasattr(X_sample, "iloc") else X_sample[0]
        shap_value_instance = shap_array[0] if isinstance(shap_array, np.ndarray) else shap_array[0]

        # 4. Force Plot
        try:
            force_path = f'plots/shap_force_model{model_index}_{model_name}.html'
            shap.initjs()
            force_plot = shap.force_plot(explainer.expected_value, shap_value_instance, instance, feature_names=feature_names, show=False, matplotlib=False)
            shap.save_html(force_path, force_plot)
            print(f"✅ SHAP force plot saved to {os.path.abspath(force_path)}")
        except Exception as e:
            print(f"❌ Could not create force plot: {e}")

        # 5. Waterfall Plot
        try:
            plt.figure(figsize=(10, 8))
            shap.plots.waterfall(shap_values[0], show=False)
            plt.tight_layout()
            waterfall_path = f'plots/shap_waterfall_model{model_index}_{model_name}.png'
            plt.savefig(waterfall_path)
            plt.show()  # This ensures it's visible inline
            plt.close()
            print(f"✅ SHAP waterfall plot saved to {os.path.abspath(waterfall_path)}")
        except Exception as e:
            print(f"❌ Could not create waterfall plot: {e}")

        print("\n✅ SHAP analysis completed successfully!")
        return True

    except Exception as e:
        print(f"\n❌ SHAP analysis failed: {e}")
        print("\n⚠️ Trying PyCaret's built-in SHAP plot as fallback...")
        try:
            from pycaret.regression import plot_model
            plot_model(tuned_model, plot='shap', save=True)
            print("✅ SHAP plot created using PyCaret's built-in functionality")
            return True
        except Exception as e2:
            print(f"❌ PyCaret's SHAP plot also failed: {e2}")
            print("\n💡 Recommendations:")
            print("   1. Check if your model is compatible with SHAP")
            print("   2. Try running just the model importance plot: plot_model(tuned_model, plot='feature')")
            print("   3. Make sure your model is properly trained and accessible")
            print("   4. Verify that you have the latest versions of SHAP and matplotlib")
            return False



# Use this to check if your plots directory exists and what's in it
def check_outputs():
    print("\nChecking for output files:")
    try:
        if os.path.exists('plots'):
            files = os.listdir('plots')
            if files:
                print(f"✅ Found {len(files)} files in plots directory:")
                for file in files:
                    print(f"   - {file}")
            else:
                print("⚠️ 'plots' directory exists but is empty")
        else:
            print("❌ 'plots' directory does not exist")
    except Exception as e:
        print(f"❌ Error checking outputs: {e}")

# First check if required packages are installed
if check_and_install_packages():
    # If you have a trained model from PyCaret, use it like this:
    from pycaret.regression import load_model, setup, get_config
    
    # Load your dataset and set up PyCaret (replace with your actual code)
    # df = pd.read_csv('your_dataset.csv')
    # setup(df, target='your_target_column')
    
    # Either load an existing model or use your already trained model
    """
    try:
        # If you have a saved model:
        #model = load_model('your_saved_model_path')
        print("Model loaded successfully!")
    except:
        print("Please replace 'your_saved_model_path' with your actual model path")
        print("Or use your existing trained model variable")
        # model = your_existing_trained_model_variable
    """
    
    # Call the SHAP analysis function with your model
    # run_shap_analysis(model, model_index=1, debug=True)

# -------- RUN SHAP FOR ALL TUNED MODELS --------
for idx, tuned_model in enumerate(tuned_models, 1):
    run_shap_analysis(tuned_model, model_index=idx, debug=True)

# Optionally, check plot outputs
check_outputs()

In [ ]:
# Extract feature importance directly (if available)

from pycaret.regression import get_config
import pandas as pd
import numpy as np
import os

os.makedirs('data', exist_ok=True)  # Ensure output directory exists

for idx, tuned_model in enumerate(tuned_models, 1):
    model_name = type(tuned_model).__name__
    print(f"\nModel {idx}: {model_name}")

    try:
        feature_names = get_config('X_transformed').columns

        if hasattr(tuned_model, 'feature_importances_'):
            importances = tuned_model.feature_importances_
            importance_label = 'feature_importances_'
        elif hasattr(tuned_model, 'coef_'):
            importances = np.abs(tuned_model.coef_)
            importance_label = 'coef_ (abs)'
        else:
            print("Feature importance attribute not available for this model.")
            continue

        # Ensure lengths match
        if len(feature_names) != len(importances):
            print(f"Warning: Length mismatch - {len(feature_names)} features vs {len(importances)} importance values")
            min_length = min(len(feature_names), len(importances))
            feature_names = feature_names[:min_length]
            importances = importances[:min_length]

        fi = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        })
        fi = fi.sort_values('importance', ascending=False)
        print(fi.head(15))  # Show top 15 features

        # Save to CSV with model index and name
        out_path = f"data/feature_importance_model_{idx}_{model_name}.csv"
        fi.to_csv(out_path, index=False)
        print(f"Feature importance ({importance_label}) saved to '{out_path}'")
    except Exception as e:
        print(f"Failed to extract feature importance for {model_name}: {e}")